## Installing the required packages

In [ ]:
%pip install datasets transformers evaluate jiwer

In [ ]:
import torch
import datasets
import evaluate
import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import '+', TrainingArguments, Trainer

Explore the Data online [here](https://huggingface.co/datasets/SPRINGLab/asr-task-data).

In [ ]:
dataset = datasets.load_dataset("SPRINGLab/asr-task-data")

This function takes a batch of data as input. Each batch contains text samples. It concatenates all the text samples in the batch into a single string `all_text`. It then creates a list of unique characters `vocab` from this combined text and returns a dictionary with two keys: `vocab`, containing the list of unique characters, and `all_text`, containing the combined text.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocabs = dataset.map(extract_all_chars, batch_size=8)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

`vocab_list` is initialized to hold all characters found across the dataset.
We iterate over the vocabularies from each batch in the training set and extends `vocab_list` with the characters from each batch's vocabulary.

In [ ]:
vocab_list = []

In [ ]:
for v in vocabs["train"]["vocab"]:
  vocab_list.extend(v[0])

This converts vocab_list into a set to remove any duplicate characters and then back into a list.

In [ ]:
vocab_list = list(set(vocab_list))

This creates a dictionary vocab_dict where each unique character is mapped to a unique integer ID as tokens, starting from 0.

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

This merges the mapping for the space character " " with the pipe character "|". The space character is deleted from the dictionary after its ID is reassigned to the pipe character.   

To make it clearer that " " has its own token class, we give it a more visible character "|". In addition, we also add an "[UNK]" token so that the model can later deal with characters not encountered in the training set.

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

35

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

A `Wav2Vec2CTCTokenizer` is created using the saved vocab.json file.  

It defines the following special tokens:  
unk_token: "[UNK]" for unknown characters.  
pad_token: "[PAD]" for padding.  
word_delimiter_token: "|", which is used as the token for separating words (since spaces were replaced by pipes earlier).   

[🤗 Doc](https://huggingface.co/docs/transformers/en/model_doc/wav2vec2#transformers.Wav2Vec2CTCTokenizer) for Wav2Vec2 Tokenizer.

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

The `Wav2Vec2FeatureExtractor` is initialized to process audio features.

Key parameters:  
feature_size=1: Indicates 1D audio features.  
sampling_rate=16000: Audio is expected to have a sampling rate of 16,000 Hz.   
padding_value=0.0: Padding values are set to 0.0.  
do_normalize=True: Normalization of audio is enabled.  
return_attention_mask=False: Attention masks are not used.  

[🤗 Docs](https://huggingface.co/docs/transformers/en/model_doc/wav2vec2#transformers.Wav2Vec2FeatureExtractor) for Wav2Vec2FeatureExtractor

In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

`Wav2Vec2Processor` combines feature extractor and tokenizer.  

[🤗 Docs](https://huggingface.co/docs/transformers/en/model_doc/wav2vec2#transformers.Wav2Vec2Processor) for Wav2Vec2Processor

In [ ]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
print("Target text:", dataset["train"][56]["text"])
print("Input array shape:", np.asarray(dataset["train"][56]["audio"]["array"]).shape)
print("Sampling rate:", dataset["train"][56]["audio"]["sampling_rate"])

Target text: the first location within my training data set now in order to do it for training i would
Input array shape: (76624,)
Sampling rate: 16000


This function prepares the data for training by extracting features.

In [ ]:
def prepare_dataset(batch):
    audio = batch["audio"]
    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
dataset = dataset.map(prepare_dataset, batch_size=8)

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

We define a error metric and function to compute it while training.

In [ ]:
wer_metric = evaluate.load("wer")

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
dataset = dataset['train'].train_test_split(test_size=0.1, shuffle=True, seed=42)

Base Pretrained model is defined and parameters are set for training.  

Refer to [🤗 Trainer](https://huggingface.co/docs/transformers/en/main_classes/trainer) and [🤗 TrainingArguments](https://huggingface.co/docs/transformers/v4.44.2/en/main_classes/trainer#transformers.TrainingArguments) to learn about the Parameters.

In [ ]:
model = AutoModelForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_encoder()

In [ ]:
training_args = TrainingArguments(
  output_dir="AsrTaskModel",
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=7,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=1000,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
  load_best_model_at_end=True,
  save_strategy="steps",
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.575200,3.073219,1.000000
1000,1.920800,0.961972,0.556434


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
500,3.575200,3.073219,1.000000
1000,1.920800,0.961972,0.556434
1500,0.851400,0.712503,0.445027
2000,0.667200,0.614925,0.351242
2500,0.548200,0.602136,0.351586
3000,0.468000,0.499315,0.291498
3500,0.440400,0.499604,0.280667
4000,0.384000,0.514028,0.273618
4500,0.361700,0.449441,0.262013
5000,0.305000,0.470037,0.257801


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:157: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of 